In [1]:
import os
import json
import random
from datetime import datetime, timedelta

# Configuration
NUM_FILES = 5000
CITIES = [f"City_{i}" for i in range(100, 200)]
NULL_PROBABILITY = 0.005  # Adjust to between 0.5% - 0.1%
FILE_SIZE_RANGE = (50, 100)
OUTPUT_DIR = "C:/Users/rajashekarm/Downloads/Life Science/Personal_Medicine"


In [2]:
def generate_flight_record():
    # Randomly assign some values
    record = {
        "date": (datetime.now() + timedelta(days=random.randint(-365, 365))).isoformat(),
        "origin_city": random.choice(CITIES),
        "destination_city": random.choice(CITIES),
        "flight_duration_secs": random.randint(3600, 7200),  # 1 to 2 hours
        "num_passengers": random.randint(50, 300)
    }
    # Introduce NULLs in some records
    if random.random() < NULL_PROBABILITY:
        key = random.choice(list(record.keys()))
        record[key] = None
    return record

def generate_files():
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    for _ in range(NUM_FILES):
        num_records = random.randint(*FILE_SIZE_RANGE)
        origin_city = random.choice(CITIES)
        date_stamp = datetime.now().strftime("%m-%y")
        filename = f"{OUTPUT_DIR}/{date_stamp}-{origin_city}-flights.json"
        
        flights = [generate_flight_record() for _ in range(num_records)]
        
        with open(filename, "w") as file:
            json.dump(flights, file)

# Run the generator
generate_files()


In [3]:
import time
import glob
from collections import defaultdict
import numpy as np

In [4]:
def process_files():
    start_time = time.time()
    total_records, dirty_records = 0, 0
    durations_by_city = defaultdict(list)
    passengers_in = defaultdict(int)
    passengers_out = defaultdict(int)

    for filepath in glob.glob(f"{OUTPUT_DIR}/*.json"):
        with open(filepath, "r") as file:
            flights = json.load(file)
            total_records += len(flights)
            
            for flight in flights:
                # Count dirty records
                if None in flight.values():
                    dirty_records += 1

                # Flight duration analysis
                if flight["destination_city"] and flight["flight_duration_secs"]:
                    durations_by_city[flight["destination_city"]].append(flight["flight_duration_secs"])
                
                # Track passengers
                if flight["origin_city"] and flight["num_passengers"]:
                    passengers_out[flight["origin_city"]] += flight["num_passengers"]
                if flight["destination_city"] and flight["num_passengers"]:
                    passengers_in[flight["destination_city"]] += flight["num_passengers"]

    # Calculate flight duration stats
    top_25_cities = sorted(durations_by_city, key=lambda k: len(durations_by_city[k]), reverse=True)[:25]
    duration_stats = {
        city: {
            "avg_duration": np.mean(durations_by_city[city]),
            "p95_duration": np.percentile(durations_by_city[city], 95)
        }
        for city in top_25_cities
    }

    # Find cities with max passengers arrived and departed
    max_in_city = max(passengers_in, key=passengers_in.get)
    max_out_city = max(passengers_out, key=passengers_out.get)

    end_time = time.time()
    run_duration = end_time - start_time
    
    return {
        "total_records": total_records,
        "dirty_records": dirty_records,
        "run_duration": run_duration,
        "duration_stats": duration_stats,
        "max_in_city": (max_in_city, passengers_in[max_in_city]),
        "max_out_city": (max_out_city, passengers_out[max_out_city])
    }

# Run processing
results = process_files()
print(results)


{'total_records': 7421, 'dirty_records': 36, 'run_duration': 0.11423921585083008, 'duration_stats': {'City_150': {'avg_duration': 5411.791666666667, 'p95_duration': 6982.25}, 'City_145': {'avg_duration': 5311.852631578948, 'p95_duration': 6949.9}, 'City_104': {'avg_duration': 5558.7692307692305, 'p95_duration': 7017.5}, 'City_116': {'avg_duration': 5450.9670329670325, 'p95_duration': 7013.0}, 'City_130': {'avg_duration': 5253.318681318681, 'p95_duration': 6900.5}, 'City_155': {'avg_duration': 5560.472527472528, 'p95_duration': 7086.0}, 'City_184': {'avg_duration': 5509.825581395349, 'p95_duration': 7076.25}, 'City_113': {'avg_duration': 5418.453488372093, 'p95_duration': 6982.25}, 'City_158': {'avg_duration': 5361.882352941177, 'p95_duration': 6950.2}, 'City_182': {'avg_duration': 5547.364705882353, 'p95_duration': 7088.4}, 'City_124': {'avg_duration': 5326.976190476191, 'p95_duration': 6970.65}, 'City_134': {'avg_duration': 5295.428571428572, 'p95_duration': 7036.65}, 'City_175': {'av

In [5]:
import pandas as pd

def process_files():
    start_time = time.time()
    total_records, dirty_records = 0, 0
    durations_by_city = defaultdict(list)
    passengers_in = defaultdict(int)
    passengers_out = defaultdict(int)

    # Processing files
    for filepath in glob.glob(f"{OUTPUT_DIR}/*.json"):
        with open(filepath, "r") as file:
            flights = json.load(file)
            total_records += len(flights)
            
            for flight in flights:
                # Count dirty records
                if None in flight.values():
                    dirty_records += 1

                # Flight duration analysis
                if flight["destination_city"] and flight["flight_duration_secs"]:
                    durations_by_city[flight["destination_city"]].append(flight["flight_duration_secs"])
                
                # Track passengers
                if flight["origin_city"] and flight["num_passengers"]:
                    passengers_out[flight["origin_city"]] += flight["num_passengers"]
                if flight["destination_city"] and flight["num_passengers"]:
                    passengers_in[flight["destination_city"]] += flight["num_passengers"]

    # Calculate flight duration stats for top 25 cities by number of records
    top_25_cities = sorted(durations_by_city, key=lambda k: len(durations_by_city[k]), reverse=True)[:25]
    duration_stats = {
        city: {
            "avg_duration": np.mean(durations_by_city[city]),
            "p95_duration": np.percentile(durations_by_city[city], 95)
        }
        for city in top_25_cities
    }

    # Find cities with max passengers arrived and departed
    max_in_city = max(passengers_in, key=passengers_in.get)
    max_out_city = max(passengers_out, key=passengers_out.get)

    end_time = time.time()
    run_duration = end_time - start_time

    # DataFrames for each category of results
    summary_df = pd.DataFrame({
        "total_records": [total_records],
        "dirty_records": [dirty_records],
        "run_duration": [run_duration]
    })

    duration_stats_df = pd.DataFrame.from_dict(duration_stats, orient="index")
    duration_stats_df.index.name = "destination_city"
    duration_stats_df.reset_index(inplace=True)

    passengers_in_df = pd.DataFrame.from_dict(passengers_in, orient="index", columns=["passengers_in"])
    passengers_in_df.index.name = "city"
    passengers_in_df.reset_index(inplace=True)

    passengers_out_df = pd.DataFrame.from_dict(passengers_out, orient="index", columns=["passengers_out"])
    passengers_out_df.index.name = "city"
    passengers_out_df.reset_index(inplace=True)

    max_passenger_df = pd.DataFrame({
        "city": [max_in_city, max_out_city],
        "type": ["max_in_city", "max_out_city"],
        "passengers": [passengers_in[max_in_city], passengers_out[max_out_city]]
    })

    # Return all DataFrames
    return {
        "summary": summary_df,
        "duration_stats": duration_stats_df,
        "passengers_in": passengers_in_df,
        "passengers_out": passengers_out_df,
        "max_passenger_summary": max_passenger_df
    }

# Run processing and store results in DataFrames
results = process_files()

# Display the DataFrames
print("Summary:\n", results["summary"])
print("\nDuration Stats:\n", results["duration_stats"])
print("\nPassengers In:\n", results["passengers_in"])
print("\nPassengers Out:\n", results["passengers_out"])
print("\nMax Passenger Summary:\n", results["max_passenger_summary"])


Summary:
    total_records  dirty_records  run_duration
0           7421             36        1.2667

Duration Stats:
    destination_city  avg_duration  p95_duration
0          City_150   5411.791667       6982.25
1          City_145   5311.852632       6949.90
2          City_104   5558.769231       7017.50
3          City_116   5450.967033       7013.00
4          City_130   5253.318681       6900.50
5          City_155   5560.472527       7086.00
6          City_184   5509.825581       7076.25
7          City_113   5418.453488       6982.25
8          City_158   5361.882353       6950.20
9          City_182   5547.364706       7088.40
10         City_124   5326.976190       6970.65
11         City_134   5295.428571       7036.65
12         City_175   5234.892857       6854.95
13         City_100   5388.500000       7058.45
14         City_170   5568.457831       7025.60
15         City_133   5436.409639       6969.60
16         City_111   5286.939024       7062.50
17         City_

In [6]:
import os

def process_files():
    start_time = time.time()
    total_records, dirty_records = 0, 0
    durations_by_city = defaultdict(list)
    passengers_in = defaultdict(int)
    passengers_out = defaultdict(int)

    # Processing files
    for filepath in glob.glob(f"{OUTPUT_DIR}/*.json"):
        with open(filepath, "r") as file:
            flights = json.load(file)
            total_records += len(flights)
            
            for flight in flights:
                # Count dirty records
                if None in flight.values():
                    dirty_records += 1

                # Flight duration analysis
                if flight["destination_city"] and flight["flight_duration_secs"]:
                    durations_by_city[flight["destination_city"]].append(flight["flight_duration_secs"])
                
                # Track passengers
                if flight["origin_city"] and flight["num_passengers"]:
                    passengers_out[flight["origin_city"]] += flight["num_passengers"]
                if flight["destination_city"] and flight["num_passengers"]:
                    passengers_in[flight["destination_city"]] += flight["num_passengers"]

    # Calculate flight duration stats for top 25 cities by number of records
    top_25_cities = sorted(durations_by_city, key=lambda k: len(durations_by_city[k]), reverse=True)[:25]
    duration_stats = {
        city: {
            "avg_duration": np.mean(durations_by_city[city]),
            "p95_duration": np.percentile(durations_by_city[city], 95)
        }
        for city in top_25_cities
    }

    # Find cities with max passengers arrived and departed
    max_in_city = max(passengers_in, key=passengers_in.get)
    max_out_city = max(passengers_out, key=passengers_out.get)

    end_time = time.time()
    run_duration = end_time - start_time

    # DataFrames for each category of results
    summary_df = pd.DataFrame({
        "total_records": [total_records],
        "dirty_records": [dirty_records],
        "run_duration": [run_duration]
    })

    duration_stats_df = pd.DataFrame.from_dict(duration_stats, orient="index")
    duration_stats_df.index.name = "destination_city"
    duration_stats_df.reset_index(inplace=True)

    passengers_in_df = pd.DataFrame.from_dict(passengers_in, orient="index", columns=["passengers_in"])
    passengers_in_df.index.name = "city"
    passengers_in_df.reset_index(inplace=True)

    passengers_out_df = pd.DataFrame.from_dict(passengers_out, orient="index", columns=["passengers_out"])
    passengers_out_df.index.name = "city"
    passengers_out_df.reset_index(inplace=True)

    max_passenger_df = pd.DataFrame({
        "city": [max_in_city, max_out_city],
        "type": ["max_in_city", "max_out_city"],
        "passengers": [passengers_in[max_in_city], passengers_out[max_out_city]]
    })

    # Ensure output directory exists
    os.makedirs(OUTPUT_DIR, exist_ok=True)

    # Save each DataFrame to CSV files
    summary_df.to_csv(os.path.join(OUTPUT_DIR, "summary.csv"), index=False)
    duration_stats_df.to_csv(os.path.join(OUTPUT_DIR, "duration_stats.csv"), index=False)
    passengers_in_df.to_csv(os.path.join(OUTPUT_DIR, "passengers_in.csv"), index=False)
    passengers_out_df.to_csv(os.path.join(OUTPUT_DIR, "passengers_out.csv"), index=False)
    max_passenger_df.to_csv(os.path.join(OUTPUT_DIR, "max_passenger_summary.csv"), index=False)

    # Return all DataFrames (optional)
    return {
        "summary": summary_df,
        "duration_stats": duration_stats_df,
        "passengers_in": passengers_in_df,
        "passengers_out": passengers_out_df,
        "max_passenger_summary": max_passenger_df
    }

# Run processing and save results
results = process_files()


In [7]:
git init

SyntaxError: invalid syntax (2830201818.py, line 1)